# "On the Spot Recruitment" Challenge - h2o.ai Version

for fun.

http://www.h2o.ai/

In [29]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*DisplayFormatter.*')
warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*metadata*')
warnings.simplefilter('ignore')

import h2o
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.transforms.decomposition import H2OPCA
from h2o.grid.grid_search import H2OGridSearch
from h2o.transforms.preprocessing import H2OScaler
from sklearn.pipeline import Pipeline

# Below is my exploratory code with h2o's platform

In [30]:
h2o.init()

H2O cluster uptime:,1 minutes 59 seconds 68 milliseconds
H2O cluster version:,3.8.2.9
H2O cluster name:,H2O_started_from_python_dgonzalez_esg210
H2O cluster total nodes:,1
H2O cluster total free memory:,1.76 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [3]:
df = pd.read_csv('challenge.csv') #imported with pandas

In [4]:
df = h2o.H2OFrame(df) #df is now a h2o.frame.H2OFrame


Parse Progress: [##################################################] 100%


In [5]:
df.types #returns a dictionary of dtypes

{'C1': 'enum',
 'C10': 'int',
 'C11': 'int',
 'C12': 'int',
 'C13': 'int',
 'C14': 'int',
 'C15': 'int',
 'C16': 'int',
 'C17': 'int',
 'C18': 'int',
 'C19': 'int',
 'C2': 'int',
 'C3': 'int',
 'C4': 'int',
 'C5': 'int',
 'C6': 'int',
 'C7': 'int',
 'C8': 'int',
 'C9': 'int'}

In [6]:
df.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19
B,1312,1445,882,nan,609,241,180,394,782,249,227,531,153,88,1086,131,179,591
B,1439,1503,985,230,629,222,150,416,824,233,248,632,141,100,1158,148,181,643
B,1378,1464,934,nan,620,233,135,403,778,244,240,575,144,107,1131,116,169,610
B,1315,1367,895,230,564,207,158,394,801,224,242,568,116,79,1090,132,189,594
B,1413,1500,969,nan,645,247,161,426,823,241,252,607,120,99,1175,131,197,654
B,1090,1195,740,nan,493,189,122,350,673,234,185,462,188,90,901,101,138,476
B,1294,1421,872,239,606,226,155,396,780,237,238,577,149,101,1084,124,168,578
B,1377,1504,954,248,660,240,159,417,812,240,245,614,128,91,1149,129,175,628
B,1296,1439,878,208,630,215,nan,387,759,248,219,584,151,117,1069,121,159,578
B,1470,1563,987,236,672,231,185,429,856,227,268,659,103,94,1240,132,196,683


In [7]:
df.columns #list of column names

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19']

In [8]:
df.describe()

Rows:148 Cols:19

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C1N,1-Byte Integers (w/o NAs),4,21.052631,864 B,16.0
C1S,1-Byte Fractions,7,36.842106,1.6 KB,30.074076
C2,2-Byte Integers,8,42.105263,2.8 KB,53.925926



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
127.0.0.1:54321,5.3 KB,148.0,1.0,19.0
mean,5.3 KB,148.0,1.0,19.0
min,5.3 KB,148.0,1.0,19.0
max,5.3 KB,148.0,1.0,19.0
stddev,0 B,0.0,0.0,0.0
total,5.3 KB,148.0,1.0,19.0


,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19
type,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int
mins,0.0,1030.0,1121.0,665.0,172.0,434.0,141.0,121.0,303.0,640.0,190.0,173.0,435.0,13.0,60.0,856.0,101.0,132.0,473.0
mean,NaN,1490.047619047619,1557.8287671232872,1020.7210884353733,256.91129032258056,662.9183673469389,232.78378378378378,179.6530612244898,441.3716216216217,876.6326530612243,239.3648648648648,271.7931034482758,650.9489051094889,123.4864864864865,94.51351351351349,1247.1029411764705,138.972972972973,195.94594594594588,698.7905405405406
maxs,2.0,1893.0,1945.0,1315.0,332.0,893.0,308.0,299.0,547.0,1090.0,290.0,371.0,798.0,216.0,137.0,1568.0,169.0,271.0,880.0
sigma,NaN,165.0343813632889,153.62575931583237,124.31757732390021,32.15865448808662,88.82247264420903,29.573738244409522,27.33296796590142,42.67808314610587,76.45592172991306,16.870917958745746,36.535921546040015,67.84501381478002,41.444904249515595,14.548294654947117,145.16041085700428,13.821971046790406,22.754167976957298,77.54360225090119
zeros,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
missing,0,1,2,1,24,1,0,1,0,1,0,3,11,0,0,12,0,0,0
0,B,1312.0,1445.0,882.0,nan,609.0,241.0,180.0,394.0,782.0,249.0,227.0,531.0,153.0,88.0,1086.0,131.0,179.0,591.0
1,B,1439.0,1503.0,985.0,230.0,629.0,222.0,150.0,416.0,824.0,233.0,248.0,632.0,141.0,100.0,1158.0,148.0,181.0,643.0
2,B,1378.0,1464.0,934.0,nan,620.0,233.0,135.0,403.0,778.0,244.0,240.0,575.0,144.0,107.0,1131.0,116.0,169.0,610.0


In [9]:
y = df['C1'] #response
X = df[1:] #explanatory

In [10]:
X.head()

C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19
1312,1445,882,nan,609,241,180,394,782,249,227,531,153,88,1086,131,179,591
1439,1503,985,230,629,222,150,416,824,233,248,632,141,100,1158,148,181,643
1378,1464,934,nan,620,233,135,403,778,244,240,575,144,107,1131,116,169,610
1315,1367,895,230,564,207,158,394,801,224,242,568,116,79,1090,132,189,594
1413,1500,969,nan,645,247,161,426,823,241,252,607,120,99,1175,131,197,654
1090,1195,740,nan,493,189,122,350,673,234,185,462,188,90,901,101,138,476
1294,1421,872,239,606,226,155,396,780,237,238,577,149,101,1084,124,168,578
1377,1504,954,248,660,240,159,417,812,240,245,614,128,91,1149,129,175,628
1296,1439,878,208,630,215,nan,387,759,248,219,584,151,117,1069,121,159,578
1470,1563,987,236,672,231,185,429,856,227,268,659,103,94,1240,132,196,683


In [11]:
X.isna()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
means = X.mean(na_rm=True) #ignores missing values
means

[1490.047619047619,
 1557.8287671232872,
 1020.7210884353733,
 256.91129032258056,
 662.9183673469389,
 232.78378378378378,
 179.6530612244898,
 441.3716216216217,
 876.6326530612243,
 239.3648648648648,
 271.7931034482758,
 650.9489051094889,
 123.4864864864865,
 94.51351351351349,
 1247.1029411764705,
 138.972972972973,
 195.94594594594588,
 698.7905405405406]

In [13]:
medians = []
for i in range(0,18):
    medians.append(X[i].median(na_rm=True)) #did not end up using this. 
medians

[1486.0,
 1566.5,
 1016.0,
 256.0,
 669.0,
 233.5,
 179.0,
 440.0,
 879.0,
 239.0,
 268.0,
 650.0,
 125.0,
 94.5,
 1241.5,
 138.0,
 194.5,
 700.0]

In [14]:
X.impute(method='mean')

[1490.047619047619,
 1557.8287671232872,
 1020.7210884353733,
 256.91129032258056,
 662.9183673469389,
 232.78378378378378,
 179.6530612244898,
 441.3716216216217,
 876.6326530612243,
 239.3648648648648,
 271.7931034482758,
 650.9489051094889,
 123.4864864864865,
 94.51351351351349,
 1247.1029411764705,
 138.972972972973,
 195.94594594594588,
 698.7905405405406]

In [15]:
X #imputed version

C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19
1312,1445,882,256.911,609,241,180,394,782,249,227,531,153,88,1086,131,179,591
1439,1503,985,230,629,222,150,416,824,233,248,632,141,100,1158,148,181,643
1378,1464,934,256.911,620,233,135,403,778,244,240,575,144,107,1131,116,169,610
1315,1367,895,230,564,207,158,394,801,224,242,568,116,79,1090,132,189,594
1413,1500,969,256.911,645,247,161,426,823,241,252,607,120,99,1175,131,197,654
1090,1195,740,256.911,493,189,122,350,673,234,185,462,188,90,901,101,138,476
1294,1421,872,239,606,226,155,396,780,237,238,577,149,101,1084,124,168,578
1377,1504,954,248,660,240,159,417,812,240,245,614,128,91,1149,129,175,628
1296,1439,878,208,630,215,179.653,387,759,248,219,584,151,117,1069,121,159,578
1470,1563,987,236,672,231,185,429,856,227,268,659,103,94,1240,132,196,683


# Boosting Model

In [16]:
gbm_classifier = H2OGradientBoostingEstimator(distribution="multinomial",
                                              ntrees=10,max_depth=3,min_rows=2, learn_rate="0.2")

gbm_classifier.train(x = list(range(0,df.ncol)), y = 0, training_frame=df)
gbm_classifier


gbm Model Build Progress: [##################################################] 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1474952103556_19

Model Summary: 


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,30.0,4169.0,3.0,3.0,3.0,4.0,8.0,6.6




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.08316947254818259
R^2: 0.8743624740210074
LogLoss: 0.315916847608469
Mean Per-Class Error: 0.01361111111111111

Confusion Matrix: vertical: actual; across: predicted



A,B,C,Error,Rate
50.0,0.0,0.0,0.0,0 / 50
0.0,49.0,1.0,0.02,1 / 50
0.0,1.0,47.0,0.0208333,1 / 48
50.0,50.0,48.0,0.0135135,2 / 148



Top-3 Hit Ratios: 


k,hit_ratio
1,0.9864865
2,1.0
3,1.0



Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_classification_error
,2016-09-26 21:55:09,0.026 sec,0.0,0.4444444,1.0986123,0.6621622
,2016-09-26 21:55:09,0.264 sec,1.0,0.3408391,0.8791112,0.1418919
,2016-09-26 21:55:09,0.341 sec,2.0,0.2719254,0.7394747,0.0878378
,2016-09-26 21:55:09,0.430 sec,3.0,0.2265139,0.6453041,0.0743243
,2016-09-26 21:55:09,0.484 sec,4.0,0.1862562,0.5591657,0.0472973
,2016-09-26 21:55:09,0.526 sec,5.0,0.1618055,0.5046177,0.0405405
,2016-09-26 21:55:09,0.579 sec,6.0,0.1383540,0.4506939,0.0472973
,2016-09-26 21:55:09,0.637 sec,7.0,0.1214615,0.4118247,0.0202703
,2016-09-26 21:55:09,0.694 sec,8.0,0.1082188,0.3796919,0.0202703
,2016-09-26 21:55:09,0.733 sec,9.0,0.0940347,0.3442954,0.0135135



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
C6,70.6627350,1.0,0.3607418
C17,27.1561871,0.3843070,0.1386356
C19,23.5305691,0.3329983,0.1201264
C10,10.3912268,0.1470538,0.0530485
C3,10.2216339,0.1446538,0.0521827
C13,9.9656916,0.1410318,0.0508761
C14,8.9982367,0.1273406,0.0459371
C4,7.2969480,0.1032644,0.0372518
C8,7.2818999,0.1030515,0.0371750
C11,6.4512215,0.0912959,0.0329343


# GLM Model

In [17]:
glm_classifier = H2OGeneralizedLinearEstimator(family='multinomial',nfolds=10,alpha=0.5)

glm_classifier.train(x = list(range(0,df.ncol)), y = 0, training_frame=df)
glm_classifier


glm Model Build Progress: [##################################################] 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1474952103556_20

GLM Model: summary



,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,multinomial,multinomial,"Elastic Net (alpha = 0.5, lambda = 0.04012 )",26,25,4,Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex




ModelMetricsMultinomialGLM: glm
** Reported on train data. **

MSE: 0.1930651348851813

ModelMetricsMultinomialGLM: glm
** Reported on cross-validation data. **

MSE: 0.22759936635933162

Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid,cv_7_valid,cv_8_valid,cv_9_valid,cv_10_valid
accuracy,0.7225245,0.0558805,0.7647059,0.8333333,0.7727272,0.7058824,0.7142857,0.8,0.5555556,0.7692308,0.6666667,0.6428571
err,0.2774755,0.0558805,0.2352941,0.1666667,0.2272727,0.2941177,0.2857143,0.2,0.4444444,0.2307692,0.3333333,0.3571429
err_count,4.0,0.7071068,4.0,2.0,5.0,5.0,4.0,3.0,4.0,3.0,5.0,5.0
logloss,0.6414289,0.0568314,0.6316798,0.6169704,0.5693035,0.6647027,0.6229448,0.6615686,0.7419282,0.4603127,0.6961394,0.7487394
max_per_class_error,0.5133333,0.0616441,0.5,0.4,0.3333333,0.5,0.6,0.5,0.6,0.6,0.6,0.5
mean_per_class_accuracy,0.746455,0.0445630,0.7222222,0.8666667,0.7518519,0.7111111,0.7333333,0.8333333,0.6888889,0.8,0.6666667,0.6904762
mean_per_class_error,0.253545,0.0445630,0.2777778,0.1333333,0.2481481,0.2888889,0.2666667,0.1666667,0.3111111,0.2,0.3333333,0.3095238
mse,0.2298366,0.0250584,0.2319537,0.2123435,0.1955877,0.2380138,0.2221435,0.2377744,0.2761230,0.1531090,0.2536177,0.2776995
null_deviance,32.87954,5.129859,37.416115,26.740288,49.33454,37.427773,30.831757,33.16268,20.324003,29.484701,32.96522,31.10831
r2,0.616846,0.0831948,0.623401,0.7451878,0.5848051,0.6301829,0.651679,0.6335668,0.4114221,0.8297669,0.6195735,0.4388753



Scoring History: 


,timestamp,duration,iteration,negative_log_likelihood,objective
,2016-09-26 21:55:11,0.000 sec,0,162.5674675,1.0984288
,2016-09-26 21:55:11,0.050 sec,1,88.9180843,0.6007979
,2016-09-26 21:55:11,0.078 sec,2,83.2995310,0.5628347
,2016-09-26 21:55:11,0.099 sec,3,82.6801741,0.5586498
,2016-09-26 21:55:11,0.111 sec,4,82.6773918,0.5586310


# Exploring their PCA

In [18]:
pca_decomp = H2OPCA(k=5,transform='NONE',pca_method='Power')

pca_decomp.train(x=list(range(0,df.ncol)), training_frame=df)
pca_decomp


pca Model Build Progress: [##################################################] 100%
Model Details
H2OPCA :  Principle Components Analysis
Model Key:  PCA_model_python_1474952103556_21

Importance of components: 


,pc1,pc2,pc3,pc4,pc5
Standard deviation,3177.2205776,74.0680013,58.7497084,43.7513140,27.5038836
Proportion of Variance,0.9985367,0.0005427,0.0003414,0.0001893,0.0000748
Cumulative Proportion,0.9985367,0.9990793,0.9994208,0.9996101,0.9996849




ModelMetricsPCA: pca
** Reported on train data. **

MSE: NaN


In [19]:
pred = pca_decomp.predict(df)
pred


pca prediction Progress: [##################################################] 100%


PC1,PC2,PC3,PC4,PC5
-2805.51,2.86701,98.3157,-7.69655,32.3764
-3003.66,0.444448,36.4814,30.2217,22.054
-2893.14,-15.6014,63.795,-2.65308,46.4688
-2770.73,31.9089,9.457,5.89576,7.57311
-2998.21,-13.0644,35.404,-1.35239,7.9889
-2342.5,60.765,119.744,-13.2811,46.2827
-2782.1,7.2219,92.1635,-17.9184,3.12292
-2959.08,-28.2102,71.9888,-5.99236,-4.17153
-2784.48,-18.2403,115.834,-4.85936,3.30229
-3118.36,-24.9275,14.2153,-14.3515,-20.847


# Their GridSearch is FAST, as expected

In [20]:
ntrees_opt = [5,10,15,20]
max_depth_opt = [2,3,4,5]
learn_rate_opt = [0.1,0.2,0.3]

hyper_params = {'ntrees': ntrees_opt, 'max_depth': max_depth_opt, 'learn_rate':learn_rate_opt}

gs = H2OGridSearch(H2OGradientBoostingEstimator(distribution='multinomial'), hyper_params = hyper_params)

gs.train(x = list(range(0,df.ncol)), y = 0, training_frame=df)


gbm Grid Build Progress: [##################################################] 100%


In [21]:
print (gs.sort_by('logloss',increasing=True))


Grid Search Results for H2OGradientBoostingEstimator: 


Model Id,"Hyperparameters: [max_depth, ntrees, learn_rate]",logloss
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_47,"[5, 20, 0.3]",0.0449732
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_44,"[4, 20, 0.3]",0.0680021
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_35,"[5, 15, 0.3]",0.0860526
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_46,"[5, 20, 0.2]",0.0989656
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_41,"[3, 20, 0.3]",0.1138612
---,---,---
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_9,"[5, 5, 0.1]",0.6656456
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_12,"[2, 10, 0.1]",0.6694059
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_6,"[4, 5, 0.1]",0.6724359
Grid_GBM_Key_Frame__upload_bf47352c8f276c064849bece141f4000.hex_model_python_1474952103556_23_model_3,"[3, 5, 0.1]",0.7161444



See the whole table with table.as_data_frame()



# From the above results, we build a GBoosting Model

In [22]:
gbm_classifier = H2OGradientBoostingEstimator(distribution="multinomial",ntrees=20,max_depth=5,learn_rate="0.3")

gbm_classifier.train(x = list(range(0,df.ncol)), y = 0, training_frame=df)
gbm_classifier


gbm Model Build Progress: [##################################################] 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1474952103556_24

Model Summary: 


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,60.0,9432.0,4.0,5.0,4.983333,6.0,13.0,8.216666




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.004180160362194533
R^2: 0.9936853632707925
LogLoss: 0.04497321641456268
Mean Per-Class Error: 0.0

Confusion Matrix: vertical: actual; across: predicted



A,B,C,Error,Rate
50.0,0.0,0.0,0.0,0 / 50
0.0,50.0,0.0,0.0,0 / 50
0.0,0.0,48.0,0.0,0 / 48
50.0,50.0,48.0,0.0,0 / 148



Top-3 Hit Ratios: 


k,hit_ratio
1,1.0
2,1.0
3,1.0



Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_classification_error
,2016-09-26 21:55:17,0.021 sec,0.0,0.4444444,1.0986123,0.6621622
,2016-09-26 21:55:17,0.235 sec,1.0,0.2849025,0.7655876,0.1148649
,2016-09-26 21:55:17,0.366 sec,2.0,0.2005944,0.5910355,0.0945946
,2016-09-26 21:55:17,0.445 sec,3.0,0.1459302,0.4685123,0.0608108
,2016-09-26 21:55:17,0.568 sec,4.0,0.1130085,0.3899381,0.0270270
---,---,---,---,---,---,---
,2016-09-26 21:55:19,1.925 sec,16.0,0.0104763,0.0765134,0.0
,2016-09-26 21:55:19,2.026 sec,17.0,0.0078580,0.0659389,0.0
,2016-09-26 21:55:19,2.262 sec,18.0,0.0065699,0.0579187,0.0
,2016-09-26 21:55:19,2.269 sec,19.0,0.0052206,0.0517656,0.0



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
C6,48.9293747,1.0,0.3052671
C17,21.0651455,0.4305215,0.1314240
C3,12.6970091,0.2594967,0.0792158
C10,12.6764822,0.2590771,0.0790877
C19,11.0012932,0.2248403,0.0686363
C13,7.9040785,0.1615406,0.0493130
C14,7.5658331,0.1546276,0.0472027
C7,6.7103696,0.1371440,0.0418655
C8,6.5150094,0.1331513,0.0406467
C4,5.5548463,0.1135278,0.0346563


# Now we make a Prediction

In [23]:
example = h2o.H2OFrame([np.nan, 1115, 1566.5, 748, 182, 669.0, 233.5, 178, 311, 756, 226, 268.0, 650.0, 125.0, 48, 1009, 138.0, 204, 593])

gbm_classifier.predict(example[1:])


Parse Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%


predict,A,B,C
B,0.0220014,0.656162,0.321837


# Building a pipeline

In [24]:
pipeline = Pipeline([('standardize',H2OScaler()),('pca',H2OPCA(k=2)),
                     ('gbm',H2OGradientBoostingEstimator(distribution='multinomial'))])

pipeline.fit(df[1:], df[0])


pca Model Build Progress: [##################################################] 100%

pca prediction Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%
Model Details
H2OPCA :  Principle Components Analysis
Model Key:  PCA_model_python_1474952103556_25

Importance of components: 


,pc1,pc2
Standard deviation,43938.3964330,7403.2441099
Proportion of Variance,0.9446650,0.0268184
Cumulative Proportion,0.9446650,0.9714835




ModelMetricsPCA: pca
** Reported on train data. **

MSE: NaN
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1474952103556_26

Model Summary: 


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,150.0,24879.0,4.0,5.0,4.9866667,6.0,12.0,8.866667




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.08249740563151993
R^2: 0.8753777122101508
LogLoss: 0.2914402263659982
Mean Per-Class Error: 0.06861111111111111

Confusion Matrix: vertical: actual; across: predicted



...


Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_classification_error
,2016-09-26 21:55:21,0.004 sec,0.0,0.4444444,1.0986123,0.6824324
,2016-09-26 21:55:21,0.075 sec,1.0,0.4036471,1.0099925,0.2297297
,2016-09-26 21:55:21,0.145 sec,2.0,0.3691969,0.9388854,0.2297297
,2016-09-26 21:55:21,0.151 sec,3.0,0.3410737,0.8824943,0.2297297
,2016-09-26 21:55:21,0.154 sec,4.0,0.3175395,0.8357434,0.2297297
---,---,---,---,---,---,---
,2016-09-26 21:55:21,0.417 sec,46.0,0.0896736,0.3093305,0.0743243
,2016-09-26 21:55:21,0.420 sec,47.0,0.0873134,0.3040129,0.0608108
,2016-09-26 21:55:21,0.423 sec,48.0,0.0857290,0.2996699,0.0675676
,2016-09-26 21:55:21,0.430 sec,49.0,0.0841522,0.2955575,0.0675676



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PC2,235.1169281,1.0,0.6915370
PC1,104.8748779,0.4460541,0.3084630


Pipeline(steps=[('standardize', <h2o.transforms.preprocessing.H2OScaler object at 0x11753bcc0>), ('pca', ), ('gbm', )])

# I wanted to explore a little more of their capabilities but ran into this error. Bug perhaps?

In [25]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from sklearn.grid_search import RandomizedSearchCV
from h2o.cross_validation import H2OKFold
from h2o.model.regression import h2o_r2_score
from sklearn.metrics.scorer import make_scorer

params = {'standardize_center': [True, False],
         'standardize_scale':[True, False],
         'pca_k':[2,3],
         'gbm_ntrees':[10,20],
         'gbm_max_depth':[1,2,3],
         'gbm_learn_rate':[0.1,0.2]}

# Keys just don't show up

In [26]:
H2OGradientBoostingEstimator().get_params().keys() 

dict_keys([])

In [27]:
custom_cv = H2OKFold(df, n_folds=5, seed = 42)

pipeline = Pipeline([('standardize',H2OScaler()),
                     ('pca',H2OPCA()),
                     ('gbm',H2OGradientBoostingEstimator(distribution='multinomial'))])

random_search = RandomizedSearchCV(pipeline, params, n_iter=5, scoring=make_scorer(h2o_r2_score),
                                  cv=custom_cv, random_state=42, n_jobs=1)

In [28]:
random_search.fit(df[1:],df[0])

ValueError: Invalid parameter standardize_center for estimator Pipeline. Check the list of available parameters with `estimator.get_params().keys()`.